## Steps
1. Reproducing kaggle https://www.kaggle.com/code/jamesloy/deep-learning-based-recommender-systems
2. Recreate all models in neural collaborative filtering

In [1]:
import os

os.chdir("/Users/yenchenchou/Documents/GitHub/mle-career-path")

In [2]:
import random
import polars as pl
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets

In [3]:
from ml.recsys.movielens_data import EnvInit, DataDownloader, DataCleaner, FeatEngineer

In [4]:
env_init = EnvInit()
seed = env_init.fix_seed(seed=12345)
device = env_init.available_device()

In [5]:
downloader = DataDownloader()
cleaner = DataCleaner()
featengineer = FeatEngineer()
df = downloader()
df = cleaner(df)
df = featengineer(df)
df.head()

/Users/yenchenchou/Documents/GitHub/mle-career-path/ml/recsys/movielens_data.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  pd.read_table(


user_id,movie_id,rating,timestamp
i32,i32,i32,i64
1,3186,4,978300019
1,1270,5,978300055
1,1721,4,978300055
1,1022,5,978300055
1,2340,3,978300103


In [13]:
df[["user_id", "movie_id"]].unique()

user_id,movie_id
i32,i32
1,914
1,1287
1,531
1,1545
1,1566
2,1259
2,1188
2,1792
2,2881


In [10]:
for u, i in df[["user_id", "movie_id"]].unique().iter_rows():
    print(u, i)

1 3408
1 260
1 938
1 1097
1 914
1 661
1 531
1 2791
2 2194
2 647
2 1188
2 3107
2 292
1 1022
1 720
1 1962
1 150
1 2918
1 1029
1 588
1 1
2 1357
2 1687
1 1270
1 2340
1 2018
1 3114
1 2294
2 1217
2 1213
2 3147
2 1537
2 3257
2 434
1 1193
1 2692
2 3468
2 1610
2 2881
1 3186
1 527
2 1253
2 3071
2 2236
2 1792
1 1207
1 608
1 2797
1 745
2 1210
2 3068
2 3255
2 2916
1 919
1 1246
1 1566
1 1907
2 3035
2 3256
1 1836
1 2028
1 1035
1 1287
1 2687
1 48
2 110
2 3108
2 2490
2 1544
1 2804
1 1028
1 2321
1 594
1 595
2 3030
1 1197
1 1545
1 2355
2 3105
2 902
2 2278
2 1690
2 2126
1 3105
2 1293
2 3471
2 2268
2 368
1 1721
1 1961
1 2762
1 2398
1 783
2 1834
2 1259
2 1103
2 3578
2 589
2 648
2 2628


In [22]:
df["movie_id"].unique().to_numpy().flatten()

array([   1,   48,  110,  150,  260,  292,  368,  434,  527,  531,  588,
        589,  594,  595,  608,  647,  648,  661,  720,  745,  783,  902,
        914,  919,  938, 1022, 1028, 1029, 1035, 1097, 1103, 1188, 1193,
       1197, 1207, 1210, 1213, 1217, 1246, 1253, 1259, 1270, 1287, 1293,
       1357, 1537, 1544, 1545, 1566, 1610, 1687, 1690, 1721, 1792, 1834,
       1836, 1907, 1961, 1962, 2018, 2028, 2126, 2194, 2236, 2268, 2278,
       2294, 2321, 2340, 2355, 2398, 2490, 2628, 2687, 2692, 2762, 2791,
       2797, 2804, 2881, 2916, 2918, 3030, 3035, 3068, 3071, 3105, 3107,
       3108, 3114, 3147, 3186, 3255, 3256, 3257, 3408, 3468, 3471, 3578],
      dtype=int32)

In [ ]:
# Get a list of all movie IDs
 = featengineer.uniq_items

# Placeholfers that will hold the training data
users, items, labels = [], [], []

# This is the set of items that each usesr has interaction with
user_item_set = set(zip(train_ratings['userId'],train_ratings['movieId']))

# 4:1 ratio of negative to positive samples
num_negatives = 4

for (u, i) in tqdm(user_item_set):
    users.append(u)
    items.append(i)
    labels.append(1) # items that the user has interacted with are positive
    for _ in range(num_negatives):
        # randomly select an item
        negative_item = np.random.choice(all_movieIds)
        # check that the user has not interacted with this item
        while (u, negative_item) in user_item_set:
            negative_item = np.random.choice(all_movieIds)
        users.append(u)
        items.append(negative_item)
        labels.append(0) # items not interacted with are negative

In [ ]:
import polars as pl
import numpy as np
from tqdm import tqdm

# Assuming 'ratings' and 'train_ratings' are Polars DataFrames
# Convert Pandas DataFrame to Polars DataFrame if needed
# ratings = pl.from_pandas(ratings)
# train_ratings = pl.from_pandas(train_ratings)

# Get a list of all unique movie IDs
all_movie_ids = featengineer.uniq_items

# Create a DataFrame from the user-item interactions
user_item_df = train_ratings.select(['userId', 'movieId']).distinct()

# 4:1 ratio of negative to positive samples
num_negatives = 4

# Initialize lists to store the training data
users, items, labels = [], [], []

for row in tqdm(user_item_df.to_numpy()):
    u, i = row
    users.append(u)
    items.append(i)
    labels.append(1)  # Positive sample

    # Generate negative samples
    negatives = np.random.choice(all_movie_ids, num_negatives * 2)  # Buffer to reduce likelihood of retries
    neg_count = 0
    for neg in negatives:
        if neg_count >= num_negatives:
            break
        if not user_item_df.filter((pl.col('userId') == u) & (pl.col('movieId') == neg)).to_numpy().size:
            users.append(u)
            items.append(neg)
            labels.append(0)  # Negative sample
            neg_count += 1

# Convert lists to Polars DataFrame
training_data = pl.DataFrame({
    'userId': users,
    'movieId': items,
    'label': labels
})
